# TensorFlow Basics & Convolution

TensorFlow does not do computation immediately but constructs a graph. We define everything that we want to compute, in a graph and running it requires a session.

### Intro

In [90]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [91]:
x = np.linspace(-3, 3, 5) # Computes values immediately
print x

[-3.  -1.5  0.   1.5  3. ]


LinSpace:0 means output of LinSpace. TensorFlow doesn't compute the values immediately. It only specifies the nature of the output of a TF operation, also called an Op node.

In [94]:
x = tf.linspace(-3.0, 3.0, 100) # Doesn't compute immediately
# Note that tf.linspace(-3, 3, 5) gives an error because datatypes are
# mismatched
print (x)

Tensor("LinSpace_11:0", shape=(100,), dtype=float32)


We can get the elements of the graph by doing as follows. We can also get the output of a certain node in the graph

In [95]:
g = tf.get_default_graph()
print [op.name for op in g.get_operations()] # List of ops
print g.get_tensor_by_name('LinSpace_1:0')
# Note that LinSpace has a :0 at the end of it. 
# Without :0, it refers to the Node itself, with :0 it refers to the 
# Tensor

[u'LinSpace/start', u'LinSpace_1/start', u'LinSpace_1/stop', u'LinSpace_1/num', u'LinSpace_1', u'LinSpace_2/start', u'LinSpace_2/stop', u'LinSpace_2/num', u'LinSpace_2', u'LinSpace_3/start', u'LinSpace_3/stop', u'LinSpace_3/num', u'LinSpace_3', u'LinSpace_4/start', u'LinSpace_4/stop', u'LinSpace_4/num', u'LinSpace_4', u'LinSpace_5/start', u'LinSpace_5/stop', u'LinSpace_5/num', u'LinSpace_5', u'LinSpace_6/start', u'LinSpace_6/stop', u'LinSpace_6/num', u'LinSpace_6', u'LinSpace_7/start', u'LinSpace_7/stop', u'LinSpace_7/num', u'LinSpace_7', u'Sqrt/x', u'Sqrt', u'mul/y', u'mul', u'div/x', u'div', u'sub/y', u'sub', u'Pow/y', u'Pow', u'Pow_1/x', u'Pow_1/y', u'Pow_1', u'mul_1/x', u'mul_1', u'div_1', u'mul_2/x', u'mul_2', u'Exp', u'mul_3', u'Sqrt_1/x', u'Sqrt_1', u'mul_4/y', u'mul_4', u'div_2/x', u'div_2', u'sub_1/y', u'sub_1', u'Pow_2/y', u'Pow_2', u'Pow_3/x', u'Pow_3/y', u'Pow_3', u'mul_5/x', u'mul_5', u'div_3', u'mul_6/x', u'mul_6', u'Exp_1', u'mul_7', u'Sqrt_2/x', u'Sqrt_2', u'mul_8/y', u

### Session
In order to get run a TF program, we need a session. The session computes the graph we construct. Here's an example.

In [96]:
sess = tf.Session()

# We can ask a session to compute the value of a node
computed_x = sess.run(x)
# print (computed_x)

# Or we can ask the node to compute itself using the session
computed_x = x.eval(session = sess)

# print computed_x

# We can close the session by doing this
sess.close()

We can ask TF to create a new graph and have it be connected to another session. We are allowed to have multiple sessions running at the same time.

In [97]:
g = tf.get_default_graph() # Fetch the default graph
g2 = tf.Graph()
print g2
sess2 = tf.Session(graph = g2)
print sess2

##### Interactive Session - This is a way to run session in environments like notebooks where you don't want to pass around a session variable. But it's just like a session. Here's how to create one. Also this behaves more like a normal python program. You have to recompute the formula if you want updates. For example, z is defined below in the gaussian curve example. You have to rerun the formula after changing x to get new z. Just running z.eval() won't do it. However, in a normal session, it will.

In [98]:
sess = tf.InteractiveSession()
# print x.eval()

In [99]:
print x.get_shape() # x.shape
print x.get_shape().as_list() # x.shape.tolist()

(100,)
[100]


### Example - Creating a Gaussian Curve

In [100]:
mean = 0
sigma = 1.0
z = 1.0/(tf.sqrt(2*3.14)*sigma) * (tf.exp(-1*(tf.pow(x-mean, 2)/(2*tf.pow(sigma, 2)))))

res = z.eval() # Note that x is already defined from above
plt.plot(res)
plt.show()

### Making it into a 2D Gaussian

In [101]:
l = z.get_shape().as_list()[0]
res2d = tf.matmul(tf.reshape(z, [l, 1]), tf.reshape(z, [1, l])).eval()
plt.imshow(res2d)
plt.show()

# Convolution

Loading 'camera' images from sklearn

In [102]:
from skimage import data
img = data.camera().astype(np.float32)

In [103]:
plt.imshow(img, cmap='gray')
plt.show()

#### Convolution operation in TF takes in a 4d tensor for images. The dimensions are (Batch x Height x Width x Channel). Our image is grayscale. So we reshape it using numpy into 4d as shown below.

In [104]:
# Image shape is 512x512
img4d = tf.reshape(img, [1, img.shape[0], img.shape[1], 1])
print img4d.get_shape()

(1, 512, 512, 1)


#### For the convolution operation we need to provide the specifics of the kernels - Height x Width x Channel x Number of kernels. Let's now convert our gaussian kernel in this format and convolve our image.

In [105]:
l = res2d.shape[0]
kernel = tf.reshape(res2d, [l, l, 1, 1])
print kernel.get_shape()

(100, 100, 1, 1)


In [106]:
# Convolution operation
convolved = tf.nn.conv2d(img4d, kernel, strides = [1, 1, 1, 1],
                        padding = 'SAME')

In [109]:
plt.imshow(convolved.eval()[0, :, :, 0], cmap = 'gray')
plt.show()